In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

## Determining the rock type boundaries

In [42]:
import pandas as pd
from quick_pp.rock_type import calc_fzi, rock_typing

core_data = pd.read_csv(r'data\01_raw\HW_core_data_all.csv')
# core_data.drop(columns=['RRT'], inplace=True)
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']

core_data = core_data[core_data['K mD'] > 0]
core_data.drop_duplicates(subset=['CPORE', 'CPERM', 'SW'], keep='last', inplace=True)
core_data['FZI'] = calc_fzi(core_data['CPERM'], core_data['CPORE'])
core_data.dropna(inplace=True)

fzi_cutoffs = [0.332, 0.662, 1.4, 3.724]
core_data['ROCK_FLAG'] = rock_typing(core_data['FZI'], fzi_cutoffs)

## Calculate J

In [ ]:
from quick_pp.core_calibration import leverett_j, poroperm_xplot

ift = 485
theta = 130

core_data['J'] = leverett_j(core_data['PC'], ift, theta, core_data['CPERM'], core_data['CPORE'])
poroperm_xplot(core_data.CPORE, core_data.CPERM)

In [ ]:
from quick_pp.core_calibration import fit_j_curve, j_xplot

for rt, data in core_data.groupby('Sample'):
    # print(rt)
    data = data.sort_values(by='J')
    a, b = fit_j_curve(data['SW'], data['J'])
    j_xplot(data['SW'], data['J'], a=a, b=b)  #, ylim=(0.01, 120))
    # break
# fit_j_curve(core_data['SW'], core_data['J'])
# j_xplot(core_data['SW'], core_data['J'])

In [ ]:
data

## Building Rock Type and FZI model to propagate to non-cored intervals

In [ ]:
import pickle
from quick_pp.rock_type import train_rock_type, train_fzi

train_data = all_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'ROCK_FLAG'])
rt_model = train_rock_type(train_data[['GR', 'NPHI', 'RHOB']], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\rt_model.qppm', 'wb') as file:
    pickle.dump(rt_model, file)


train_data = all_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'FZI'])
fzi_model = train_fzi(train_data[['GR', 'NPHI', 'RHOB']], train_data['FZI'], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'wb') as file:
    pickle.dump(fzi_model, file)

## Predict PERM and plot the results

In [ ]:
from quick_pp.plotter import plotly_log
from quick_pp.rock_type import calc_fzi_perm

# Plot individual results
well_data = all_data[all_data.WELL_NAME == 'HW-29'].copy()
well_data['ROCK_FLAG'] = rt_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
well_data['FZI'] = fzi_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
well_data['PERM'] = well_data.apply(lambda row: calc_fzi_perm(row['FZI'], row['PHIE']), axis=1)
fig = plotly_log(well_data, '')
fig.show(config=dict(scrollZoom=True))